In [ ]:
!pip install datasets transformers huggingface_hub
!pip install accelerate -U
!apt-get install git-lfs

import torch
import pandas as pd
import transformers
from datasets import Dataset
from sklearn.model_selection import train_test_split

from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, AdamW, get_linear_schedule_with_warmup
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer


torch.cuda.is_available()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.


True

In [ ]:
path = "/content/IMDB Dataset.csv"
df = pd.read_csv(path)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df.dropna(inplace = True)
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

In [ ]:
def preprocess_text(html_text):
    soup = BeautifulSoup(html_text, "html.parser")
    review = soup.get_text()
    review = re.sub(r'\[[^]]*\]', ' ', review)
    review = re.sub(r'[^a-zA-Z]', ' ', review)
    review = review.split()
    return ' '.join(review)

In [ ]:
df['review'] = df['review'].apply(preprocess_text)

<ipython-input-6-b4d15d81e83b>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html_text, "html.parser")


In [ ]:
df['sentiment'] = df['sentiment'].map({'positive':1,'negative':0})

In [ ]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production The filming tech...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there s a family where a little boy ...,0
4,Petter Mattei s Love in the Time of Money is a...,1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,Bad plot bad dialogue bad acting idiotic direc...,0
49997,I am a Catholic taught in parochial elementary...,0
49998,I m going to have to disagree with the previou...,0


In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2)  # Adjust the test_size as needed

train_data_dict = {
    "text": train_df["review"].tolist(),
    "label": train_df["sentiment"].tolist(),
}

test_data_dict = {
    "text": test_df["review"].tolist(),
    "label": test_df["sentiment"].tolist(),
}

train_dataset = Dataset.from_dict(train_data_dict)
test_dataset = Dataset.from_dict(test_data_dict)

In [ ]:
train_dataset[0]

{'text': 'Dirty Dancing follows the story of Frances Baby Houseman Jennifer Grey and her family as they spend the summer at Kellerman s Family Summer Camp We get to see her discover herself as she falls in love with the dance instructor Johnny Castle Patrick Swayze The movie is full of wonderful dancing hence the title some great music and terrific scenery It handles some tough issues with dignity and grace and of course has shocking spoiler here a happy wonderful feel good ending You know how it s going to end from the beginning but that s really not the point here The point is the journey a journey that touches every member of the family It s one of the movies I need to see whenever it comes on TV never mind the fact that I already have it memorized',
 'label': 1}

In [ ]:
test_dataset[0]

{'text': 'Lynne Ramsey makes arresting images and Samantha Morton can summon feeling with a gesture So what a drag to discover their talents wasted on this mannered pretentious lark Ramsey can t bring Callar to life Her attempts are too arty and oblique Repeatedly her camera lingers on long silent shots of the agonizing actress as if Morton s obliterated gaze alone could supply character We are in a blank Warholian hell of self indulgence for a film that has minutes to spare on bugs crawling across the floor you might think it could get round to fleshing out its protagonist But how will it do so if she rarely speaks Without the novel s interior monologue the celluloid Morvern Callar is nobody Small wonder Ramsey has Morton undress often That said the first ten minutes were so impressively acted shot and edited that my hopes were soaring Give the film that much it knows how to make promises if not how to keep any',
 'label': 0}

In [ ]:
model_name = "bert-base-uncased"  # You can choose a different BERT model as well
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_function(data):
    return tokenizer(data["text"], padding="max_length", truncation=True)




In [ ]:
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
tokenized_train

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 40000
})

In [ ]:
from datasets import load_metric

def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [ ]:
from transformers import TrainingArguments, Trainer
from accelerate import Accelerator
print(TrainingArguments)
# Define a TrainingArguments object
training_args = TrainingArguments(
    per_device_train_batch_size=16,
    output_dir="./sentiment_finetuned_model",
    evaluation_strategy="steps",
    eval_steps=500,
    save_total_limit=2,
    num_train_epochs=1,
)

# Define an Accelerator to enable mixed-precision training and distributed training
accelerator = Accelerator()



<class 'transformers.training_args.TrainingArguments'>


In [ ]:

import numpy as np
# Define a Trainer using the TrainingArguments and Accelerator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
     compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()

# Save the fine-tuned model
model.save_pretrained("./sentiment_finetuned_model")
tokenizer.save_pretrained("./sentiment_finetuned_model")


Step,Training Loss,Validation Loss,Accuracy,F1
500,0.335800,0.421374,0.851700,0.869787
1000,0.247000,0.196646,0.926000,0.928016
1500,0.223100,0.190677,0.934500,0.935360
2000,0.205300,0.191696,0.932500,0.931976
2500,0.197000,0.168304,0.939600,0.940210


<ipython-input-17-b571dbad15c2>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


('./sentiment_finetuned_model/tokenizer_config.json',
 './sentiment_finetuned_model/special_tokens_map.json',
 './sentiment_finetuned_model/vocab.txt',
 './sentiment_finetuned_model/added_tokens.json')